# Lightkurve Workshop

## Getting Started

If you would like to get started with this tutorial, we recommend you have the following installed:
* lightkurve version 1.0b26 (use pip install lightkurve --upgrade)

Once you have looked through this workshop, we also have a full set of tutorials on all of the functionality of lightkurve available here:
* http://docs.lightkurve.org/tutorials/

## Introduction

Lightkurve is a package to help users access and process data from NASA's Kepler, K2 and TESS missions. You can use lightkurve to quickly get to work using these data sets for your own science goals.

In [3]:
import lightkurve as lk

### Data types

* Target Pixel Files (TPFs): Image files containing small "cut outs" around targets. These imagettes generally contain ~20 - 200 pixels, and so they can become quite large. TPFs are the most raw form of data you will work with.

In [4]:
tpf = lk.search_targetpixelfile('Pi Mensae', sector=1).download()
tpf.plot();

ConnectionError: HTTPSConnectionPool(host='mast.stsci.edu', port=443): Max retries exceeded with url: /portal/Mashup/Mashup.asmx/columnsconfig (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1c210c7048>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

* Light Curve Files (LCFs): These files are created by the Kepler pipeline. The pipeline converts TPFs in to light curves, and stores them as a fits file. For TESS, Kepler and K2 there are two light curves in each light curve file; a Simple Aperture Photometry (SAP) light curve and a Pre Datasearch Conditioned (PDCSAP) light curve. The PDCSAP light curves have been processed to remove some instrumental artefacts.

In [ ]:
lcf = lk.search_lightcurvefile('Pi Mensae', sector=1).download()
lcf.plot();

* Light Curves: Light curves are simply target flux as a function of time. Each light curve file (LCF) has two light curves in it. Light curves can also be built out of any time/flux data you have access to.

In [ ]:
lc = lcf.PDCSAP_FLUX
lc.plot();

The primary workflow for TESS and Kepler is to go from either TPFs or LCFs to light curves, clean and process the data, and then search for signals from e.g. transiting planets.

### Working with TPFs (User Generated Light Curves)

The most interesting data in a `KeplerTargetPixelFile` object are the `flux` and `time` values which give access to the brightness of the observed target over time. You can access the timestamps of the observations using the `time` property:

In [ ]:
tpf.time

By default, `time` is in the Kepler-specific *Barycentric Kepler Julian Day* format (BKJD). You can easily convert this into [AstroPy Time objects](http://docs.astropy.org/en/stable/time/) using the `astropy_time` property:

In [ ]:
tpf.astropy_time

Next, let's look at the actual image data, which is available via the `flux` property:

In [ ]:
tpf.flux.shape

The `flux` data is a 18104 x 21 x 11 array in units electrons/second. The first axis is the time axis, and the images themselves are 21 pixels by 11 pixels. You can use the `plot` method on the `KeplerTargetPixelFile` object to view the data.

In [5]:
tpf.plot();

NameError: name 'tpf' is not defined

You also have acces to the pipeline generated pixel mask. This mask can be used to generate a light curve, or you can create your own.

In [6]:
tpf.pipeline_mask

NameError: name 'tpf' is not defined

In [7]:
tpf.plot(aperture_mask=tpf.pipeline_mask)

NameError: name 'tpf' is not defined

The `create_threshold_mask` method will allow you to build your own aperture. You can choose a `sigma` level above the background, and light curve will automatically select a target above that threshold.

In [8]:
aper = tpf.create_threshold_mask(threshold=3)
tpf.plot(aperture_mask=aper)

NameError: name 'tpf' is not defined

Now that we have an aperture, we can sum up the flux inside the aperture and create a light curve.

In [9]:
lc = tpf.to_lightcurve(aperture_mask=aper)

NameError: name 'tpf' is not defined

In [10]:
lc

NameError: name 'lc' is not defined

`lc` is now a `TessLightCurve`, which has new properties and methods. For example, light curves also have a `plot` method:


In [11]:
lc.plot()

NameError: name 'lc' is not defined

You can interact with this object using familiar `matplotlib.pyplot` keywords and syntax. There are also `scatter` and `errorbar` plotting methods.

In [12]:
ax = lc.plot(ls='--')
lc.errorbar(ax=ax, c='grey')
lc.scatter(ax=ax, c='r')

NameError: name 'lc' is not defined

You can also use the `interact` method to interactively view the pixel data

![InteractGif](interact_gif.gif "interact")

**Remember:** You can access the pipeline corrected light curves using light curve files from MAST. These have been corrected for instrument systematics.

In [13]:
lcf.plot()

NameError: name 'lcf' is not defined

In [14]:
lcf.PDCSAP_FLUX

NameError: name 'lcf' is not defined

### Making simple corrections to light curves

Light curves can be corrected with the following methods:

 * `lc.remove_nans()`
 * `lc.remove_outliers()`
 * `lc.normalize()`
 * `lc.flatten()`
 * `lc.bin()`

**Remember:** You can read more about each method in the docs, or in our docstrings.

In [15]:
ax = lc.plot()
lc.remove_outliers(sigma=5).plot(ax=ax, c='red');

NameError: name 'lc' is not defined

In [16]:
ax = lc.plot()
lc.flatten(window_length=1501, break_tolerance=50).plot(ax=ax, c='red');

NameError: name 'lc' is not defined

In [17]:
ax = lc.plot()
lc.bin(binsize=100).plot(ax=ax, c='red');

NameError: name 'lc' is not defined

### Folding Light Curves

When looking for periodic signals, it's useful to be able to fold a light curve at a period and midpoint. Light curve lets you do this with the `fold` method on light curves.

In [18]:
period = 6.27
t0 = 1325.504

In [19]:
folded_lc = lc.fold(period, t0)
folded_lc

NameError: name 'lc' is not defined

`FoldedLightCurve` is a class similar to `LightCurve`, but it has the `phase` attribute.

In [20]:
folded_lc.phase

NameError: name 'folded_lc' is not defined

This time has been folded at the period given, and centered around t0. Plotting a folded light curve will show a plot in phase.

In [21]:
folded_lc.errorbar()

NameError: name 'folded_lc' is not defined

We see here there is a transit in the center!

### Stringing Commands together

Using light curve it is possible to string these commands together in a single "sentence". 

In [22]:
lc = lcf.PDCSAP_FLUX.remove_nans().remove_outliers().flatten(window_length=1501, break_tolerance=50).fold(period, t0)
lc.bin().errorbar()
plt.xlim(-0.1, 0.1)

NameError: name 'lcf' is not defined

### Converting light curves to other formats

When you have created a light curve you may want to share it with collaborators, or submit your light curves as High Level Science Products to the MAST archive. Lightkurve lets you do this by converting light curves to files. The following methods allow easy file sharing:

* `lc.to_pandas()`
* `lc.to_fits(PATH)`
* `lc.to_csv(PATH)`

In [23]:
lc.to_pandas().head(5)

NameError: name 'lc' is not defined

In [24]:
lc.to_fits('output.fits')

NameError: name 'lc' is not defined